In [81]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pandas_profiling
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [131]:
features = pd.read_csv('data/features.csv')
stores = pd.read_csv('data/stores.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [132]:
train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [133]:
#Merge train dataset

mergeDFtrain = pd.merge(train, features,
                  on= ['Store', 'Date'],
                  how= 'inner')
TrainStore = pd.merge(mergeDFtrain, stores,
                     on = 'Store',
                     how = 'left')


#Test dataset

mergeDFtest = pd.merge(test, features,
                  on= ['Store', 'Date'],
                  how= 'inner')
TestStore = pd.merge(mergeDFtest, stores,
                     on = 'Store',
                     how = 'left')

In [134]:
TrainStore['Date'] = pd.to_datetime(TrainStore['Date'])
TestStore['Date'] = pd.to_datetime(TestStore['Date'])

In [135]:
TrainStore = TrainStore.drop('IsHoliday_y', axis=1)
TestStore = TestStore.drop('IsHoliday_y', axis=1)

In [136]:
# Seperate numerical and categorical data

#Training dataset


num_var_train = [i for i in dict(TrainStore.dtypes) if dict(TrainStore.dtypes)[i] in ['float64','int64']] 

cat_var_train = [i for i in dict(TrainStore.dtypes) if dict(TrainStore.dtypes)[i] in ['object']] 


# Testing dataset

num_var_test = [i for i in dict(TestStore.dtypes) if dict(TestStore.dtypes)[i] in ['float64','int64']] 

cat_var_test = [i for i in dict(TestStore.dtypes) if dict(TrainStore.dtypes)[i] in ['object']] 

In [137]:
def var_summary(x):
    total_count = x.count() + x.isnull().sum()
    miss_perc = round(x.isnull().sum() / total_count * 100, 2) 
    return pd.Series([x.count(),total_count,x.isnull().sum() ,miss_perc,x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N','Total_count' ,'NMISS','Miss_perc','SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

num_summary=TrainStore[num_var_train].apply(lambda x: var_summary(x)).T
num_summary

,N,Total_count,NMISS,Miss_perc,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Store,421570.0,421570.0,0.0,0.00,9.359084e+06,22.200546,22.00000,12.785297,1.634638e+02,1.000,1.000000,3.000000,5.000000,11.000000,22.00000,33.000000,40.000000,43.000000,45.000000,45.000000
Dept,421570.0,421570.0,0.0,0.00,1.865882e+07,44.260317,37.00000,30.492054,9.297654e+02,1.000,1.000000,4.000000,7.000000,18.000000,37.00000,74.000000,92.000000,95.000000,98.000000,99.000000
Weekly_Sales,421570.0,421570.0,0.0,0.00,6.737219e+09,15981.258123,7612.03000,22711.183519,5.157979e+08,-4988.940,5.000000,59.974500,291.097000,2079.650000,7612.03000,20205.852500,42845.673000,61201.951000,106479.586000,693099.360000
Temperature,421570.0,421570.0,0.0,0.00,2.533217e+07,60.090059,62.09000,18.447931,3.403262e+02,-2.060,18.300000,27.310000,33.980000,46.680000,62.09000,74.280000,83.580000,87.270000,92.810000,100.140000
Fuel_Price,421570.0,421570.0,0.0,0.00,1.416908e+06,3.361027,3.45200,0.458515,2.102356e-01,2.472,2.565000,2.653000,2.720000,2.933000,3.45200,3.738000,3.917000,4.029000,4.202000,4.468000
MarkDown1,150681.0,421570.0,270889.0,64.26,1.091898e+09,7246.420196,5347.45000,8291.221345,6.874435e+07,0.270,17.760000,149.190000,375.200000,2240.270000,5347.45000,9210.900000,15282.470000,21801.350000,41524.030000,88646.760000
MarkDown2,111248.0,421570.0,310322.0,73.61,3.709708e+08,3334.628621,192.00000,9475.357325,8.978240e+07,-265.760,-0.990000,1.950000,6.980000,41.600000,192.00000,1926.940000,8549.740000,16497.470000,50366.600000,104519.540000
MarkDown3,137091.0,421570.0,284479.0,67.48,1.973317e+08,1439.421384,24.60000,9623.078290,9.260364e+07,-29.100,0.060000,0.650000,1.650000,5.080000,24.60000,103.990000,400.090000,1059.900000,63143.290000,141630.610000
MarkDown4,134967.0,421570.0,286603.0,67.98,4.566161e+08,3383.168256,1481.31000,6292.384031,3.959410e+07,0.220,3.970000,28.760000,108.710000,504.220000,1481.31000,3595.040000,7871.420000,12645.960000,35785.260000,67474.850000
MarkDown5,151432.0,421570.0,270138.0,64.08,7.009750e+08,4628.975079,3359.45000,5962.887455,3.555603e+07,135.160,351.970000,715.520000,1070.830000,1878.440000,3359.45000,5563.800000,8337.700000,11269.240000,27754.230000,108519.280000


In [138]:
num_sum_test = TestStore[num_var_test].apply(lambda x: var_summary(x)).T
num_sum_test

,N,Total_count,NMISS,Miss_perc,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
Store,115064.0,115064.0,0.0,0.00,2.558817e+06,22.238207,22.000000,12.809930,1.640943e+02,1.000000,1.000000,3.0000,5.000000,11.000000,22.000000,33.000000,40.000000,43.000000,45.000000,45.000000
Dept,115064.0,115064.0,0.0,0.00,5.101883e+06,44.339524,37.000000,30.656410,9.398155e+02,1.000000,1.000000,4.0000,7.000000,18.000000,37.000000,74.000000,92.000000,95.000000,98.000000,99.000000
Temperature,115064.0,115064.0,0.0,0.00,6.206760e+06,53.941804,54.470000,18.724153,3.505939e+02,-7.290000,11.440000,23.9800,29.970000,39.820000,54.470000,67.350000,79.480000,83.820000,92.140000,101.950000
Fuel_Price,115064.0,115064.0,0.0,0.00,4.121070e+05,3.581546,3.606000,0.239442,5.733244e-02,2.872000,2.957000,3.1610,3.227000,3.431000,3.606000,3.766000,3.866000,3.951000,4.079000,4.125000
MarkDown1,114915.0,115064.0,149.0,0.13,8.836063e+08,7689.216439,4842.290000,10698.760716,1.144635e+08,-2781.450000,20.840000,189.4900,457.660000,1966.460000,4842.290000,9439.140000,15325.040000,23140.880000,65757.570000,103184.980000
MarkDown2,86437.0,115064.0,28627.0,24.88,3.227602e+08,3734.051729,742.590000,8323.495014,6.928057e+07,-35.740000,0.020000,6.1400,25.770000,180.350000,742.590000,2735.670000,9277.290000,22671.670000,42920.210000,71074.170000
MarkDown3,105235.0,115064.0,9829.0,8.54,2.528890e+08,2403.088666,78.260000,13767.939313,1.895562e+08,-179.260000,0.150000,1.1800,3.100000,15.100000,78.260000,272.580000,698.930000,2361.570000,86476.660000,149483.310000
MarkDown4,102176.0,115064.0,12888.0,11.20,3.429250e+08,3356.219071,840.940000,7570.501545,5.731249e+07,0.220000,2.610000,16.9600,37.360000,155.460000,840.940000,3096.920000,7674.410000,14191.010000,44739.280000,65344.640000
MarkDown5,115064.0,115064.0,0.0,0.00,4.513594e+08,3922.681189,2390.430000,19445.150745,3.781139e+08,-185.170000,269.300000,540.8900,770.883000,1309.300000,2390.430000,4227.270000,6966.340000,9316.710000,18599.300000,771448.100000
CPI,76902.0,115064.0,38162.0,33.17,1.360868e+07,176.961347,192.304445,41.239967,1.700735e+03,131.236226,131.279355,131.4784,132.030871,138.402033,192.304445,223.244532,225.898059,227.784753,228.729864,228.976456


In [139]:
# Outlier Treatment
TrainStore.Weekly_Sales=np.where(TrainStore.Weekly_Sales>100000, 100000,TrainStore.Weekly_Sales)
TrainStore.Temperature = np.where(TrainStore.Weekly_Sales>92.810000, 92.810000,TrainStore.Temperature)


In [140]:
#Expand Date

TrainStore['Day_of_week'] = TrainStore['Date'].dt.dayofweek
TrainStore['Month'] =TrainStore['Date'].dt.month
TrainStore['Year'] = TrainStore['Date'].dt.year
TrainStore['Quarter'] = TrainStore['Date'].dt.quarter


TestStore['Day_of_week'] = TestStore['Date'].dt.dayofweek
TestStore['Month'] =TestStore['Date'].dt.month
TestStore['Year'] = TestStore['Date'].dt.year
TestStore['Quarter'] = TestStore['Date'].dt.quarter


In [141]:
TrainStore.isnull().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
Type                 0
Size                 0
Day_of_week          0
Month                0
Year                 0
Quarter              0
dtype: int64

In [142]:
#Impute missing 
TrainStore.MarkDown1 = TrainStore.MarkDown1.fillna(TrainStore.MarkDown1.mean())
TrainStore.MarkDown2 = TrainStore.MarkDown2.fillna(TrainStore.MarkDown2.mean())
TrainStore.MarkDown3 = TrainStore.MarkDown3.fillna(TrainStore.MarkDown3.mean())
TrainStore.MarkDown4 = TrainStore.MarkDown4.fillna(TrainStore.MarkDown4.mean())
TrainStore.MarkDown5 = TrainStore.MarkDown5.fillna(TrainStore.MarkDown5.mean())

In [143]:
TrainStore.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
Day_of_week     0
Month           0
Year            0
Quarter         0
dtype: int64

In [144]:
TestStore.isnull().sum()

Store               0
Dept                0
Date                0
IsHoliday_x         0
Temperature         0
Fuel_Price          0
MarkDown1         149
MarkDown2       28627
MarkDown3        9829
MarkDown4       12888
MarkDown5           0
CPI             38162
Unemployment    38162
Type                0
Size                0
Day_of_week         0
Month               0
Year                0
Quarter             0
dtype: int64

In [145]:
#impute Markdown with mean
TestStore.MarkDown1 = TestStore.MarkDown1.fillna(TestStore.MarkDown1.mean())
TestStore.MarkDown2 = TestStore.MarkDown2.fillna(TestStore.MarkDown2.mean())
TestStore.MarkDown3 = TestStore.MarkDown3.fillna(TestStore.MarkDown3.mean())
TestStore.MarkDown4 = TestStore.MarkDown4.fillna(TestStore.MarkDown4.mean())

TestStore.CPI = TestStore.CPI.fillna(TestStore.CPI.mean())
TestStore.Unemployment = TestStore.Unemployment.fillna(TestStore.Unemployment.mean())

In [146]:
TestStore.isnull().sum()

Store           0
Dept            0
Date            0
IsHoliday_x     0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
Day_of_week     0
Month           0
Year            0
Quarter         0
dtype: int64

In [147]:
TrainStore['IsHoliday_x'] = np.where(TrainStore['IsHoliday_x'] == 'True', 1 ,0)
TestStore['IsHoliday_x'] = np.where(TestStore['IsHoliday_x'] == 'True' , 1, 0)

C:\Users\Abhay\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [148]:
TrainStore['Type'] = np.where(TrainStore['Type']== 'A', 1,
                   (np.where(TrainStore['Type'] == 'B', 2, 3)))

TestStore['Type'] = np.where(TestStore['Type']=='A',1,
                            (np.where(TestStore['Type'] == 'B', 2 ,3)))

In [149]:
TrainStore.head(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day_of_week,Month,Year,Quarter
0,1,1,2010-02-05,24924.50,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
1,1,2,2010-02-05,50605.27,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
2,1,3,2010-02-05,13740.12,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
3,1,4,2010-02-05,39954.04,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
4,1,5,2010-02-05,32229.38,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1


In [150]:
TestStore.head(5)

,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day_of_week,Month,Year,Quarter
0,1,1,2012-11-02,0,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1,151315,4,11,2012,4
1,1,2,2012-11-02,0,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1,151315,4,11,2012,4
2,1,3,2012-11-02,0,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1,151315,4,11,2012,4
3,1,4,2012-11-02,0,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1,151315,4,11,2012,4
4,1,5,2012-11-02,0,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1,151315,4,11,2012,4


In [151]:

TrainStore['SuperBowl'] = np.where((TrainStore['Date']==datetime(2010,2,12)) | (TrainStore['Date']==datetime(2011,2,11)) | (TrainStore['Date']==datetime(2012,2,10)) | (TrainStore['Date']==datetime(2013,2,8)), 1,0)
TrainStore['LaborDay'] = np.where((TrainStore['Date']==datetime(2010,9,10)) | (TrainStore['Date']==datetime(2011,9,9)) | (TrainStore['Date']==datetime(2012,9,7)) | (TrainStore['Date']==datetime(2013,9,6)), 1,0)
TrainStore['Thanksgiving'] = np.where((TrainStore['Date']==datetime(2010,11,26)) | (TrainStore['Date']==datetime(2011,11,25)) | (TrainStore['Date']==datetime(2012,11,23)) | (TrainStore['Date']==datetime(2013,11,29)), 1,0)
TrainStore['Christmas'] = np.where((TrainStore['Date']==datetime(2010,12,31)) | (TrainStore['Date']==datetime(2011,12,30)) | (TrainStore['Date']==datetime(2012,12,28)) | (TrainStore['Date']==datetime(2013,12,27)), 1,0)




TestStore['SuperBowl'] = np.where((TestStore['Date']==datetime(2010,2,12)) | (TestStore['Date']==datetime(2011,2,11)) | (TestStore['Date']==datetime(2012,2,10)) | (TestStore['Date']==datetime(2013,2,8)), 1,0)
TestStore['LaborDay'] = np.where((TestStore['Date']==datetime(2010,9,10)) | (TestStore['Date']==datetime(2011,9,9)) | (TestStore['Date']==datetime(2012,9,7)) | (TestStore['Date']==datetime(2013,9,6)), 1,0)
TestStore['Thanksgiving'] = np.where((TestStore['Date']==datetime(2010,11,26)) | (TestStore['Date']==datetime(2011,11,25)) | (TestStore['Date']==datetime(2012,11,23)) | (TestStore['Date']==datetime(2013,11,29)), 1,0)
TestStore['Christmas'] = np.where((TestStore['Date']==datetime(2010,12,31)) | (TestStore['Date']==datetime(2011,12,30)) | (TestStore['Date']==datetime(2012,12,28)) | (TestStore['Date']==datetime(2013,12,27)), 1,0)

In [152]:
TrainStore['IsHoliday_x'] = TrainStore['IsHoliday_x'] | TrainStore['SuperBowl'] | TrainStore['LaborDay'] | TrainStore['Thanksgiving'] | TrainStore['Christmas']


TestStore['IsHoliday_x'] = TestStore['IsHoliday_x'] | TestStore['SuperBowl'] | TestStore['LaborDay'] | TestStore['Thanksgiving'] | TestStore['Christmas']


In [153]:
TrainStore = TrainStore.drop(columns=['SuperBowl','LaborDay','Thanksgiving','Christmas','Date'], axis = 1)
TestStore = TestStore.drop(columns=['SuperBowl','LaborDay','Thanksgiving','Christmas'], axis = 1)

In [154]:
trainX = TrainStore.drop('Weekly_Sales', axis = 1)

In [155]:
trainX

,Store,Dept,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Day_of_week,Month,Year,Quarter
0,1,1,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
1,1,2,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
2,1,3,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
3,1,4,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
4,1,5,0,92.81,2.572,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,211.096358,8.106,1,151315,4,2,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,93,0,92.81,3.882,4018.910000,58.080000,100.000000,211.940000,858.330000,192.308899,8.667,2,118221,4,10,2012,4
421566,45,94,0,92.81,3.882,4018.910000,58.080000,100.000000,211.940000,858.330000,192.308899,8.667,2,118221,4,10,2012,4
421567,45,95,0,92.81,3.882,4018.910000,58.080000,100.000000,211.940000,858.330000,192.308899,8.667,2,118221,4,10,2012,4
421568,45,97,0,92.81,3.882,4018.910000,58.080000,100.000000,211.940000,858.330000,192.308899,8.667,2,118221,4,10,2012,4


In [156]:
trainY = TrainStore.Weekly_Sales

In [157]:
# Split TrainStore into train and test dataset

train_X, test_X, train_y, test_y = train_test_split(trainX,trainY, test_size = 0.3, random_state = 123)

In [158]:
# Standardise

sc = StandardScaler()
sc_train = sc.fit_transform(train_X)
sc_test = sc.transform(test_X)

In [159]:
train_y = pd.DataFrame(train_y)

In [124]:
# RandomForest
clf = RandomForestRegressor(n_estimators=100, random_state=100)
clf.fit(sc_train,train_y)
y_pred_rf=clf.predict(sc_test)
acc_rf= clf.score(sc_train, train_y)
print ("Accuracy: %i %% \n"%acc_rf)

C:\Users\Abhay\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 0 % 



In [160]:
# Decision tree
cld=DecisionTreeRegressor()
cld.fit(sc_train, train_y)
y_pred_dt= cld.predict(sc_test)
acc_dt = round( cld.score(sc_train, train_y) * 100, 2)

print ('Accuracy :'+str(acc_dt) + '%')

Accuracy :100.0%


In [170]:
#Predicting using DT
final = pd.DataFrame({'ID' : TestStore.Store.astype(str)+'_'+TestStore.Dept.astype(str)+'_'+TestStore.Date.astype(str),
'Weekly_Sales' : cld.predict(TestStore.drop('Date', axis=1))})
final.head()

,ID,Weekly_Sales
0,1_1_2012-11-02,12165.16
1,1_2_2012-11-02,2354.48
2,1_3_2012-11-02,2354.48
3,1_4_2012-11-02,2354.48
4,1_5_2012-11-02,2354.48


In [171]:
final.to_csv('Submission.csv', index=0)